In [1]:
import numpy as np
import csv
import copy

from rcit.util.pre_process.pre_process import isolate_echo_remove
from rcit.motion.piv_motion import particle_image_velocimetry
from rcit.motion.piv_motion import burgers_equation_2D
from rcit.radar_data_io.read_dx_data import read_dwd_dx_radar_data

from rcit.util.pre_process.pre_process import input_filter


In [2]:
def write_csv(csv_path, M):
    with open(csv_path, 'w', encoding='utf-8', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(M)

In [3]:
# set forecast dataset, totally we choose 27 radar images, the first three is used for global motion calculation, 
# and the next 24 is used for 2-hr nowcasting and correspondly verification
# 2007-05-26, 0-27(00:00); 226-253(18:50); 244-271(20:20); 252-279(21:00). 
# 2008-07-19 192-219(16:00); 
# 2008-07-26 0-27(00:00); 192-219(16:00); 217-244(18:05).

f = open("./080719/ein_bild.dat")
lines = f.readlines()
lines1 = lines[192:219]
radar_images = np.full((27, 256, 256), np.nan)
radar_images_filtered = np.full((27, 256, 256), np.nan)
radar_images_intensities = np.full((27, 256, 256), np.nan)


In [4]:
# excuation of segmentation pocedure for the selected radar images
t = 0

for lines2 in lines1:
    line = lines2.split('\n')
    line1 = lines2.split('.ess\n')
    print(line1[0][2:])
    print(line[0])

    reflectivity, intensity = read_dwd_dx_radar_data("./080719/" + line[0], 6.96454, 51.40643, 152, 128000)
    origin_ref, filter_ref = input_filter(reflectivity, 'cf')
    origin_intensity, filter_intensity = input_filter(intensity, 'cf')
    filter_intensity[np.isnan(filter_intensity)] = 0
    intensity[np.isnan(intensity)] = 0
    filter_intensity_1 = copy.deepcopy(filter_intensity)
    
    radar_images[t, :, :] = reflectivity
    radar_images_filtered[t, :, :] = filter_intensity_1
    
    t += 1

0807191600
dx0807191600.ess


/Users/heting/miniconda3/envs/pyrcit/lib/python3.11/site-packages/wradlib/trafo.py:261: RuntimeWarning: overflow encountered in power
  return 10.0 ** (x / 10.0)


0807191605
dx0807191605.ess
0807191610
dx0807191610.ess
0807191615
dx0807191615.ess
0807191620
dx0807191620.ess
0807191625
dx0807191625.ess
0807191630
dx0807191630.ess
0807191635
dx0807191635.ess
0807191640
dx0807191640.ess
0807191645
dx0807191645.ess
0807191650
dx0807191650.ess
0807191655
dx0807191655.ess
0807191700
dx0807191700.ess
0807191705
dx0807191705.ess
0807191710
dx0807191710.ess
0807191715
dx0807191715.ess
0807191720
dx0807191720.ess
0807191725
dx0807191725.ess
0807191730
dx0807191730.ess
0807191735
dx0807191735.ess
0807191740
dx0807191740.ess
0807191745
dx0807191745.ess
0807191750
dx0807191750.ess
0807191755
dx0807191755.ess
0807191800
dx0807191800.ess
0807191805
dx0807191805.ess
0807191810
dx0807191810.ess


In [ ]:
input_t1 = isolate_echo_remove(radar_images_filtered[1], n=3, thr=np.min(radar_images_filtered[1])) 
input_t2 = isolate_echo_remove(radar_images_filtered[2], n=3, thr=np.min(radar_images_filtered[2]))
loc_x, loc_y, u_filt, v_filt = particle_image_velocimetry(input_t1, input_t2, 16, 16, 0.5, 0.5, 5, 5, 0.1)
print(loc_x) 
print(loc_y)

In [ ]:
Interval = 1
C = 0.1
nu = 0.2
X = loc_x
Y = loc_y
T = np.linspace(0, 23, 24)
Zx = u_filt
Zy = v_filt

write_csv("./results/init_u_motion.csv", u_filt)
write_csv("./results/init_v_motion.csv", v_filt)

u, v, w = burgers_equation_2D(X, Y, Zx, Zy, T, nu)

for i in range(24):
    write_csv("./results/motion_"+str((i+1)*5)+"min.csv", w[i])
    write_csv("./results/u_"+str((i+1)*5)+"min.csv", u[i])
    write_csv("./results/v_"+str((i+1)*5)+"min.csv", v[i])